# Interactive Jahn–Teller Effect — Teaching Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatSciEd/Materials-Chemistry/blob/main/notebooks/Electronic-Structure/Jahn-Teller%20Effect.ipynb)


## Background: What is the Jahn–Teller effect?

**Brief history.** In 1937, H. A. Jahn and E. Teller proved that any *nonlinear* molecule in an electronically **degenerate** state is unstable with respect to a nuclear distortion that removes that degeneracy. In practice, this means many **transition‑metal complexes** (and ions in crystals) spontaneously distort to lower their energy.

**Where it matters.**
- **Coordination chemistry**: e.g., octahedral **Cu(II) d<sup>9</sup>** or **Mn(III) d<sup>4</sup> (high‑spin)** show strong axial elongations/compressions.
- **Solid‑state physics**: perovskites, manganites, and cuprates exhibit cooperative JT effects that influence transport and magnetism.
- **Spectroscopy**: splittings and vibronic progressions in electronic spectra are shaped by JT coupling.

**General principles.**
- **Electronic degeneracy → vibronic coupling**: A degenerate electronic state couples to vibrational modes of matching symmetry; the simplest cartoon is a linear coupling to a normal‑mode coordinate $Q$.
- **Lowered symmetry** lifts degeneracy (e.g., $O_h \to D_{4h}$ for an axial distortion of an octahedron), stabilizing one configuration.
- **JT vs. pseudo‑JT**: True JT requires degeneracy; pseudo‑JT arises from near‑degenerate states that mix strongly via vibrations.



## Model used in this notebook

We describe the $d$-levels of an octahedral complex subject to an axial $D_{4h}$ distortion using **Ballhausen parameters** ($D_q, D_s, D_t$). The level energies (in eV) are:
$$
\begin{aligned}
\varepsilon_{a_{1g}}(d_{z^2})            &= 6Dq - 2D_s - 6D_t,\\
\varepsilon_{b_{1g}}(d_{x^2-y^2})        &= 6Dq + 2D_s - D_t,\\
\varepsilon_{b_{2g}}(d_{xy})             &= -4Dq + 2D_s - D_t,\\
\varepsilon_{e_g}(d_{xz}, d_{yz})        &= -4Dq - D_s + 4D_t.
\end{aligned}
$$

We relate the **distortion coordinate** $Q$ to $D_s$ and $D_t$ linearly:
$$
D_s = k_s Q,\qquad D_t = k_t Q,
$$
so orbital energies are (piecewise) linear in $Q$. Electrons fill these levels (0/1/2 per orbital) with a **pairing penalty** $P$ for each double occupancy. The **total energy** is
$$
E(Q)= E_{\text{elec}}(Q; P) + \tfrac12 k\,Q^2 + a_3 Q^3 + a_4 Q^4,
$$
where $k$ is a lattice stiffness; $a_3$ skews left/right (**asymmetry**); $a_4>0$ deepens the double‑well. We also provide an **optional Boltzmann soft‑minimum** $E_{\rm soft}$ to smooth sharp kinks at electronic reconfigurations.


## Parameters and controls

- **Ion / $d$-count**: pick a metal ion preset or set a custom number of $d$ electrons.
- **<i>Q</i>**: axial distortion of the octahedron; $Q>0$ elongates $z$ bonds; $Q<0$ compresses.
- **<i>D<sub>q</sub></i>**: octahedral crystal‑field splitting baseline (eV) that controls the energy gap between the two level sets $\Delta_{\text{oct}} = 10 D_q$.
- **Pairing $P$**: energy cost (eV) for doubly occupying an orbital.
- **Lattice $k$**: harmonic restoring force constant.
- **Advanced → mapping $Q \to D_s, D_t$**: slopes $k_s, k_t$.
- **Advanced → energy shaping**: $a_3$ (asymmetry) and $a_4$ (well depth).
- **Smooth energy (soft‑min)**: temperature‑like $T$ (eV).
- **Visual tuning (new)**: CH<sub>3</sub> **tilt** and **C–H length**; **arrow length** and **spacing**; **level‑label spacing** and **offset**.



## How to use the three panels

- **Atomic Structure**: Move $Q$ to elongate ($Q>0$) or compress ($Q<0$) the axial bonds. CH<sub>3</sub> are visual cues; C–H lengths are fixed in the drawing. You can adjust the **tilt angle** to approximate **tetrahedral** geometry (default ~70.5° from the M–C axis; $\cos\theta\approx 1/3$).
- **Electronic Structure**: The horizontal lines are orbital energies. A bracket on the left groups the parent $t_{2g}$ and $e_g$ sets. Right‑side labels use subscripts (e.g., $d_{xz}\,(e_g)$). Spin arrows sit **centered** on each level; in near‑degenerate groups, arrows are spread in $x$ and singles alternate left/right to avoid overlap.
- **Stability**: The energy $E(Q)$ includes electronic energy + lattice terms. Tune $k, a_3, a_4$ for single or double wells; the filled dot shows the current $Q$. Enable **Smooth energy** to round kinks from electronic reconfigurations.


In [1]:
import numpy as np
from ipywidgets import VBox, HBox, Layout, Dropdown, FloatSlider, IntSlider, Accordion, Output, Checkbox, Label
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # registers 3D projection
from IPython.display import display

plt.ioff()

def fresh_fig(figsize=(5,4)):
    fig = plt.figure(figsize=figsize)
    return fig

# --- Styling constants ---
FIG_DPI   = 120
TITLE_FS  = 16               # uniform titles
LABEL_FS  = 14               # base axis label size
TICK_FS   = 12               # base tick size

# Panel-specific font scales
P2_SCALE  = 1.15
P3_SCALE  = 1.11
P2_LABEL  = LABEL_FS * P2_SCALE
P2_TICK   = TICK_FS  * P2_SCALE
P3_LABEL  = LABEL_FS * P3_SCALE
P3_TICK   = TICK_FS  * P3_SCALE
P2_PER_LEVEL_LABEL = P2_LABEL * 0.90  # slightly smaller right-side labels

plt.rcParams.update({
    "figure.dpi": FIG_DPI,
    "axes.titlesize": TITLE_FS,
    "axes.labelsize": LABEL_FS,
    "xtick.labelsize": TICK_FS,
    "ytick.labelsize": TICK_FS
})

# --- Colors (CPK-inspired) ---
COLOR_C = '#4A4A4A'        # Carbon: dark gray
COLOR_H_FACE = '#FFFFFF'   # Hydrogen: white
COLOR_H_EDGE = '#222222'   # Hydrogen edge
COLOR_BOND_MC = '#6E6E6E'  # Metal–Carbon bond
COLOR_BOND_CH = '#BFBFBF'  # C–H bond

In [2]:

# --- Crystal-field model (exact minimization + optional soft-min smoothing) ---

ORBITALS = ['dx2-y2', 'dz2', 'dxy', 'dxz', 'dyz']

def crystal_field_energies(Dq, Ds, Dt):
    return {
        'dz2':    6*Dq - 2*Ds - 6*Dt,   # a1g
        'dx2-y2': 6*Dq + 2*Ds - Dt,     # b1g
        'dxy':   -4*Dq + 2*Ds - Dt,     # b2g
        'dxz':   -4*Dq - Ds + 4*Dt,     # eg
        'dyz':   -4*Dq - Ds + 4*Dt,     # eg
    }

def map_Q_to_DsDt(Q, ks=0.8, kt=-0.10):
    return ks*Q, kt*Q

def enumerate_occupancies(n_electrons):
    occs = []
    for o1 in range(3):
        for o2 in range(3):
            for o3 in range(3):
                for o4 in range(3):
                    for o5 in range(3):
                        if o1+o2+o3+o4+o5 == n_electrons:
                            occs.append([o1,o2,o3,o4,o5])
    return occs

def config_energy(occ_vec, energies, pairing_energy):
    E = 0.0
    n_pairs = 0
    for o, orb in zip(occ_vec, ORBITALS):
        E += o * energies[orb]
        if o == 2:
            n_pairs += 1
    return E + pairing_energy * n_pairs

def config_degeneracy(occ_vec):
    n_single = sum(1 for o in occ_vec if o == 1)
    return 2**n_single

def minimize_electronic_energy(energies, n_electrons, pairing_energy):
    best_E = None
    best_occ = None
    for occ in enumerate_occupancies(n_electrons):
        E = config_energy(occ, energies, pairing_energy)
        if (best_E is None) or (E < best_E - 1e-12):
            best_E = E
            best_occ = occ
    n_unpaired = sum(1 for o in best_occ if o == 1)
    occ_dict = {orb: o for orb, o in zip(ORBITALS, best_occ)}
    return occ_dict, best_E, n_unpaired

def softmin_electronic_energy(energies, n_electrons, pairing_energy, T_soft):
    """Stabilized soft-min using log-sum-exp trick to avoid overflow/underflow.
    Returns: -T * log(sum_g exp(-(E - E0)/T)) + E0, where E0=min(E) over configs.
    """
    if T_soft <= 0:
        return minimize_electronic_energy(energies, n_electrons, pairing_energy)[1]
    E_list = []
    g_list = []
    for occ in enumerate_occupancies(n_electrons):
        E = config_energy(occ, energies, pairing_energy)
        g = config_degeneracy(occ)
        E_list.append(E)
        g_list.append(g)
    if not E_list:
        return 0.0
    E0 = min(E_list)
    # compute Z' = sum g * exp(-(E - E0)/T)
    terms = [g*np.exp(-(E - E0)/max(T_soft, 1e-12)) for E, g in zip(E_list, g_list)]
    Zp = sum(terms) + 1e-300
    return E0 - max(T_soft, 1e-12) * np.log(Zp)

def total_energy_vs_Q(Q_grid, n_elec, Dq, P_pair, k_lattice, ks, kt, a3, a4, T_soft=0.0):
    E_tot = []
    for Q in Q_grid:
        Ds, Dt = map_Q_to_DsDt(Q, ks=ks, kt=kt)
        e = crystal_field_energies(Dq, Ds, Dt)
        if T_soft > 0:
            E_elec = softmin_electronic_energy(e, n_elec, P_pair, T_soft)
        else:
            E_elec = minimize_electronic_energy(e, n_elec, P_pair)[1]
        E_tot.append(E_elec + 0.5 * k_lattice * (Q**2) + a3*(Q**3) + a4*(Q**4))
    return np.array(E_tot)

def ion_to_dcount(label):
    mapping = {
        "Ti(III)  d1": 1,
        "V(III)   d2": 2,
        "Cr(III)  d3": 3,
        "Mn(III)  d4": 4,
        "Fe(III)  d5": 5,
        "Fe(II)   d6": 6,
        "Co(II)   d7": 7,
        "Co(III)  d6": 6,
        "Ni(II)   d8": 8,
        "Cu(II)   d9": 9,
        "Zn(II)  d10": 10,
        "Custom…": None,
    }
    return mapping.get(label, None)


In [3]:

# --- Panel 1: Atomic Structure ---

MOLECULE_SCALE = 2.0   # per user
VIEW_LIM = 1.5         # per user

def _norm(v):
    v = np.array(v, dtype=float)
    n = np.linalg.norm(v)
    return v / (n if n>1e-9 else 1.0)

def _perp_basis(v):
    v = _norm(v)
    ref = np.array([0,0,1.0]) if abs(v[2]) < 0.9 else np.array([1.0,0,0])
    u = np.cross(v, ref); u = _norm(u)
    w = np.cross(v, u);  w = _norm(w)
    return u, w

def _draw_small_cap(ax, center, vdir, ch_len, ch_tilt_deg):
    # Realistic tetrahedral-ish CH geometry:
    # theta = angle between CH bond and +vdir (outward axis)
    theta = np.deg2rad(ch_tilt_deg)
    ring_r = ch_len * np.sin(theta)
    out_s  = ch_len * np.cos(theta)
    n_subs = 3

    vhat = _norm(vdir)
    u, w = _perp_basis(vhat)
    ring_center = center + out_s * vhat
    for k in range(n_subs):
        ang = 2*np.pi * k / n_subs
        p = ring_center + ring_r * (np.cos(ang)*u + np.sin(ang)*w)
        ax.plot([center[0], p[0]], [center[1], p[1]], [center[2], p[2]],
                linewidth=1.5*MOLECULE_SCALE, color=COLOR_BOND_CH)
        ax.scatter([p[0]], [p[1]], [p[2]], s=95*(MOLECULE_SCALE**2),
                   facecolors=COLOR_H_FACE, edgecolors=COLOR_H_EDGE, linewidths=0.8)

def draw_panel1(Q, ch_len, ch_tilt_deg):
    fig = fresh_fig((5.8,5.6))
    ax = fig.add_subplot(111, projection='3d')
    r_eq = 1.0
    r_ax = 1.0 + Q
    donors = np.array([
        [ r_eq, 0.0, 0.0],
        [-r_eq, 0.0, 0.0],
        [ 0.0,  r_eq, 0.0],
        [ 0.0, -r_eq, 0.0],
        [ 0.0, 0.0,  r_ax],
        [ 0.0, 0.0, -r_ax],
    ])

    ax.scatter([0], [0], [0], s=260*(MOLECULE_SCALE**2))
    ax.scatter(donors[:,0], donors[:,1], donors[:,2], s=170*(MOLECULE_SCALE**2), c=COLOR_C)

    for p in donors:
        ax.plot([0,p[0]], [0,p[1]], [0,p[2]], linewidth=2.0*MOLECULE_SCALE, color=COLOR_BOND_MC)

    for p in donors:
        _draw_small_cap(ax, p, p - np.array([0.0,0.0,0.0]), ch_len, ch_tilt_deg)

    lim = VIEW_LIM
    ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim); ax.set_zlim(-lim, lim)
    try: ax.set_box_aspect([1,1,1])
    except Exception: pass

    try: ax.set_axis_off()
    except Exception:
        ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
        ax.set_xlabel(''); ax.set_ylabel(''); ax.set_zlabel('')
        try: ax._axis3don = False
        except Exception: pass

    ax.set_title('Molecular Structure', fontsize=TITLE_FS, y=1.02)
    plt.show()


In [4]:

# --- Panel 2: Electronic Structure (stable arrows, fixed symmetry labels) ---
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib import transforms

def _deoverlap_min_gap(sorted_items, gap):
    # Given items sorted by target y (descending), return y positions that
    # respect a minimum vertical gap without changing order.
    # Each item is (name, y0, label_text).
    ys = []
    prev = None
    for _, y0, _ in sorted_items:
        if prev is None:
            y = y0
        else:
            # enforce min gap while preserving order; top-to-bottom clamp
            y = min(y0, prev - gap)
        ys.append(y)
        prev = y
    return ys

def draw_panel2(energies, occ, n_unpaired,
                label_delta_factor=0.11, label_xoffset=0.012,
                arrow_half_factor=0.055, arrow_dx_pair=0.040):
    # Fallback fonts if globals not set
    global P2_LABEL, P2_TICK, P2_PER_LEVEL_LABEL, TITLE_FS
    try: P2_LABEL
    except NameError: P2_LABEL = 12
    try: P2_TICK
    except NameError: P2_TICK = 10
    try: P2_PER_LEVEL_LABEL
    except NameError: P2_PER_LEVEL_LABEL = 11
    try: TITLE_FS
    except NameError: TITLE_FS = 14

    # Colors (fallbacks)
    try: LEVEL_T2G
    except NameError: LEVEL_T2G = '#1f77b4'
    try: LEVEL_EG
    except NameError: LEVEL_EG = '#d62728'

    # Fixed horizontal positions for all degeneracies
    orbital_xbase = {
        'dx2-y2': 0.50,
        'dz2':    0.30,
        'dxy':    0.40,
        'dxz':    0.20,
        'dyz':    0.60,
    }

    # Taller figure so it matches/exceeds Panel 3 height
    fig, ax = plt.subplots(figsize=(7.0, 6.0))

    # Energetics
    order_desc = sorted(energies.items(), key=lambda kv: kv[1], reverse=True)
    yvals = [E for _, E in order_desc]
    y_min, y_max = min(yvals), max(yvals)
    yr = (y_max - y_min) if (y_max - y_min) > 1e-9 else 1.0

    # Asymmetric padding: more headroom at top so e_g doesn't crowd the frame
    pad_bot = 0.12 * yr
    pad_top = 0.22 * yr
    ax.set_ylim(y_min - pad_bot, y_max + pad_top)

    # If sliders exist, lock y-range across the Q range using same asymmetric padding
    try:
        Q_min = float(getattr(Q_sl, 'min', -0.8))
        Q_max = float(getattr(Q_sl, 'max',  0.8))
        ks_val = float(ks_sl.value) if 'ks_sl' in globals() else 0.8
        kt_val = float(kt_sl.value) if 'kt_sl' in globals() else -0.10
        Dq_val = float(Dq_sl.value) if 'Dq_sl' in globals() else 1.2
        def cf_energies(Dq, Q):
            Ds = ks_val * Q; Dt = kt_val * Q
            return {
                'dz2':     6*Dq - 2*Ds - 6*Dt,
                'dx2-y2':  6*Dq + 2*Ds - 1*Dt,
                'dxy':    -4*Dq + 2*Ds - 1*Dt,
                'dxz':    -4*Dq - 1*Ds + 4*Dt,
                'dyz':    -4*Dq - 1*Ds + 4*Dt,
            }
        e_min = cf_energies(Dq_val, Q_min); e_max = cf_energies(Dq_val, Q_max)
        y_floor = min(min(e_min.values()), min(e_max.values()))
        y_ceil  = max(max(e_min.values()), max(e_max.values()))
        yr_full = max(y_ceil - y_floor, 1e-6)
        pad_bot = 0.12 * yr_full
        pad_top = 0.22 * yr_full
        ax.set_ylim(y_floor - pad_bot, y_ceil + pad_top)
        yr = (y_ceil - y_floor) if (y_ceil - y_floor) > 1e-9 else 1.0
    except Exception:
        pass

    # X range and label anchor
    x_left, x_right = 0.10, 0.70
    xlim_max = 1.16
    ax.set_xlim(-0.05, xlim_max)
    x_label = min(x_right + float(label_xoffset), xlim_max - 0.02)

    # Level lines (colored by family)
    ORBITAL_GROUP = {'dx2-y2':'eg', 'dz2':'eg', 'dxy':'t2g', 'dxz':'t2g', 'dyz':'t2g'}
    for orb, E in order_desc:
        c = LEVEL_EG if ORBITAL_GROUP[orb] == 'eg' else LEVEL_T2G
        ax.plot([x_left, x_right], [E, E], color=c, linewidth=2.0)

    # Right-side labels with minimum vertical gap; dxz/dyz combined
    base_delta = float(label_delta_factor) * yr
    candidates = [
        ('dx2-y2', energies['dx2-y2'], r'$d_{x^2-y^2}\,(b_{1g})$'),
        ('dz2',    energies['dz2'],    r'$d_{z^2}\,(a_{1g})$'),
        ('dxy',    energies['dxy'],    r'$d_{xy}\,(b_{2g})$'),
        ('dxz_dyz', energies['dxz'],   r'$d_{xz},\,d_{yz}\,(e_g)$'),
    ]
    candidates.sort(key=lambda t: t[1], reverse=True)  # top to bottom
    y_clamped = _deoverlap_min_gap(candidates, base_delta)
    for (name, y0, text), ylab in zip(candidates, y_clamped):
        ax.text(x_label, ylab, text, va='center', fontsize=P2_PER_LEVEL_LABEL, clip_on=True)

    # Brackets and family labels at left
    Et = [energies['dxy'], energies['dxz'], energies['dyz']]
    Ee = [energies['dz2'],  energies['dx2-y2']]
    y0_t, y1_t = min(Et), max(Et)
    y0_e, y1_e = min(Ee), max(Ee)
    x_br, tick = x_left - 0.015, 0.015
    # t2g (blue)
    ax.plot([x_br, x_br], [y0_t, y1_t], color=LEVEL_T2G, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y0_t, y0_t], color=LEVEL_T2G, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y1_t, y1_t], color=LEVEL_T2G, linewidth=1.2)
    ax.text(x_br - 0.006, 0.5*(y0_t+y1_t), r'$t_{2g}$', ha='right', va='center',
            fontsize=P2_LABEL, color=LEVEL_T2G, clip_on=True)
    # eg (red)
    ax.plot([x_br, x_br], [y0_e, y1_e], color=LEVEL_EG, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y0_e, y0_e], color=LEVEL_EG, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y1_e, y1_e], color=LEVEL_EG, linewidth=1.2)
    ax.text(x_br - 0.006, 0.5*(y0_e+y1_e), r'$e_g$', ha='right', va='center',
            fontsize=P2_LABEL, color=LEVEL_EG, clip_on=True)

    # Electron arrows (stable positions)
    half   = float(arrow_half_factor) * yr
    dxpair = float(arrow_dx_pair)
    for (orb, E) in order_desc:
        n = occ.get(orb, 0)
        if n == 0:
            continue
        xb = orbital_xbase.get(orb, 0.40)
        if n == 1:
            ax.annotate('', xy=(xb, E + half), xytext=(xb, E - half),
                        arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))
        elif n == 2:
            xL, xR = xb - dxpair/2.0, xb + dxpair/2.0
            ax.annotate('', xy=(xL, E + half), xytext=(xL, E - half),
                        arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))
            ax.annotate('', xy=(xR, E - half), xytext=(xR, E + half),
                        arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))

    # Axes cosmetics
    ax.set_yticks(sorted(set([round(v,3) for v in energies.values()])))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.set_xticks([])
    ax.set_ylabel('Energy (eV)', fontsize=P2_LABEL)
    ax.set_xlabel(f'Levels + Spins (unpaired = {n_unpaired})', fontsize=P2_LABEL)
    ax.tick_params(axis='both', labelsize=P2_TICK)
    ax.set_title('Electronic Structure', fontsize=TITLE_FS, y=1.02)

    # Symmetry labels fixed inside frame at y=0.92
    bt = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    ax.text(x_br,    0.92, r'$O_h$',    transform=bt, ha='center', va='bottom', fontsize=P2_LABEL, fontweight='bold')
    ax.text(x_label, 0.92, r'$D_{4h}$', transform=bt, ha='center', va='bottom', fontsize=P2_LABEL, fontweight='bold')

    plt.show()


In [5]:

# --- Panel 3: Stability (match Panel 2 height) ---

def draw_panel3(Q_current, n_elec, Dq, P_pair, k_lattice, ks, kt, a3, a4,
                smooth_enabled=False, T_soft=0.02, Q_span=(-0.6, 0.6), N=241):
    fig = fresh_fig((6.0, 5.5))  # match Panel 2 size
    ax = fig.add_subplot(111)
    Q_grid = np.linspace(Q_span[0], Q_span[1], N)
    E_curve = total_energy_vs_Q(Q_grid, n_elec, Dq, P_pair, k_lattice, ks, kt, a3, a4,
                                T_soft if smooth_enabled else 0.0)
    ax.plot(Q_grid, E_curve)

    Ds, Dt = map_Q_to_DsDt(Q_current, ks=ks, kt=kt)
    e = crystal_field_energies(Dq, Ds, Dt)
    if smooth_enabled and T_soft > 0:
        E_elec = softmin_electronic_energy(e, n_elec, P_pair, T_soft)
    else:
        E_elec = minimize_electronic_energy(e, n_elec, P_pair)[1]
    E_now = E_elec + 0.5 * k_lattice * (Q_current**2) + a3*(Q_current**3) + a4*(Q_current**4)
    ax.plot(Q_current, E_now, marker='o')

    ax.set_xlabel('Distortion Q', fontsize=P3_LABEL)
    ax.set_ylabel('Total energy E(Q) (eV)', fontsize=P3_LABEL)
    ax.tick_params(axis='both', labelsize=P3_TICK)
    ax.set_title('Stability', fontsize=TITLE_FS, y=1.02)
    plt.show()


In [6]:
# --- Dashboard wiring ---

ion_dd = Dropdown(
    options=["Cu(II)   d9", "Mn(III)  d4", "Ti(III)  d1", "V(III)   d2", "Cr(III)  d3",
             "Fe(III)  d5", "Fe(II)   d6", "Co(II)   d7", "Co(III)  d6", "Ni(II)   d8",
             "Zn(II)  d10", "Custom…"],
    value="Cu(II)   d9", description="Ion:", layout=Layout(width='280px')
)
dcount_sl = IntSlider(value=9, min=0, max=10, step=1, description='d electrons:', layout=Layout(width='330px'))

Q_sl  = FloatSlider(value=0.25, min=-0.6, max=0.6, step=0.01, description='Q:', layout=Layout(width='320px'))
Dq_sl = FloatSlider(value=1.2, min=0.0, max=3.0, step=0.05, description='10Dq (eV):', layout=Layout(width='340px'))
P_sl  = FloatSlider(value=0.8, min=0.0, max=3.0, step=0.05, description='Pairing P (eV):', layout=Layout(width='340px'))
k_sl  = FloatSlider(value=2.0, min=0.0, max=6.0, step=0.1, description='Lattice k:', layout=Layout(width='320px'))

# Advanced mapping
ks_sl = FloatSlider(value=0.8,  min=-2.0, max= 2.0, step=0.05, description='k_s (Ds/Q):', layout=Layout(width='300px'))
kt_sl = FloatSlider(value=-0.10, min=-2.0, max= 2.0, step=0.05, description='k_t (Dt/Q):', layout=Layout(width='300px'))

# Advanced energy shaping
a3_sl = FloatSlider(value=-0.10, min=-3.0, max=3.0, step=0.01, description='a3 · Q^3:', layout=Layout(width='300px'))
a4_sl = FloatSlider(value= 1.20, min= 0.0, max=8.0, step=0.05, description='a4 · Q^4:', layout=Layout(width='300px'))
smooth_chk = Checkbox(value=True, description='Smooth energy (soft-min)')
Tsoft_sl   = FloatSlider(value=0.03, min=0.0, max=0.15, step=0.005, description='T (eV):', readout=True, layout=Layout(width='300px'))

# New: Visual tuning
ch_len_sl   = FloatSlider(value=0.25*MOLECULE_SCALE, min=0.18*MOLECULE_SCALE, max=0.40*MOLECULE_SCALE, step=0.01, description='C–H length', layout=Layout(width='300px'))
ch_tilt_sl  = FloatSlider(value=70.53, min=40.0, max=85.0, step=0.1, description='CH tilt (°)', layout=Layout(width='300px'))
arrow_len_sl= FloatSlider(value=0.055, min=0.030, max=0.120, step=0.002, description='Arrow half (×ΔE)', layout=Layout(width='300px'))
arrow_dx_sl = FloatSlider(value=0.040, min=0.020, max=0.080, step=0.002, description='Arrow Δx', layout=Layout(width='300px'))
label_dy_sl = FloatSlider(value=0.11,  min=0.06,  max=0.18,  step=0.005, description='Label Δy (×ΔE)', layout=Layout(width='300px'))
label_dx_sl = FloatSlider(value=0.012, min=0.006, max=0.040, step=0.001, description='Label x offset', layout=Layout(width='300px'))

adv_map_box    = VBox([ks_sl, kt_sl])
adv_energy_box = VBox([a3_sl, a4_sl, smooth_chk, Tsoft_sl])
adv_visual_box = VBox([ch_len_sl, ch_tilt_sl, Label('— Spins —'), arrow_len_sl, arrow_dx_sl, Label('— Labels —'), label_dy_sl, label_dx_sl])

acc = Accordion(children=[adv_map_box, adv_energy_box, adv_visual_box])
acc.set_title(0, 'Advanced: mapping Q → Ds, Dt')
acc.set_title(1, 'Advanced: energy shaping + smoothing')
acc.set_title(2, 'Advanced: visual tuning (CH3, arrows, labels)')
acc.layout = Layout(margin='0 0 8px 0')  # small spacer below Advanced

# Output widgets per user
out1 = Output(layout=Layout(width='360px', margin='0 6px 0 0'))
out2 = Output(layout=Layout(width='530px', margin='0 30px 0 6px'))
out3 = Output(layout=Layout(width='510px', margin='0 0 0 0'))

def on_controls_change(*args):
    d_from_ion = ion_to_dcount(ion_dd.value)
    if d_from_ion is not None:
        dcount_sl.disabled = True
        dcount_sl.value = d_from_ion
    else:
        dcount_sl.disabled = False

    Dq = Dq_sl.value
    Ds, Dt = map_Q_to_DsDt(Q_sl.value, ks=ks_sl.value, kt=kt_sl.value)
    e = crystal_field_energies(Dq, Ds, Dt)
    occ, E_elec, n_unp = minimize_electronic_energy(e, dcount_sl.value, P_sl.value)

    with out1:
        out1.clear_output(wait=True)
        draw_panel1(Q_sl.value, ch_len_sl.value, ch_tilt_sl.value)
    with out2:
        out2.clear_output(wait=True)
        draw_panel2(e, occ, n_unp,
                    label_delta_factor=label_dy_sl.value,
                    label_xoffset=label_dx_sl.value,
                    arrow_half_factor=arrow_len_sl.value,
                    arrow_dx_pair=arrow_dx_sl.value)
    with out3:
        out3.clear_output(wait=True)
        draw_panel3(Q_sl.value, dcount_sl.value, Dq_sl.value, P_sl.value,
                    k_sl.value, ks_sl.value, kt_sl.value, a3_sl.value, a4_sl.value,
                    smooth_enabled=smooth_chk.value, T_soft=Tsoft_sl.value)

for w in [ion_dd, dcount_sl, Q_sl, Dq_sl, P_sl, k_sl, ks_sl, kt_sl, a3_sl, a4_sl, smooth_chk, Tsoft_sl,
          ch_len_sl, ch_tilt_sl, arrow_len_sl, arrow_dx_sl, label_dy_sl, label_dx_sl]:
    w.observe(on_controls_change, names='value')

# Initial render
on_controls_change()

header = HBox([ion_dd, dcount_sl], layout=Layout(align_items='center', justify_content='flex-start'))
row1   = HBox([Q_sl, Dq_sl, P_sl, k_sl], layout=Layout(flex_flow='row wrap', justify_content='flex-start', gap='8px'))
panels = HBox([out1, out2, out3], layout=Layout(justify_content='flex-start', align_items='flex-start', gap='0px', margin='6px 0 0 0'))

In [7]:

# --- Presets and Reset Button ---

def apply_weak_field():
    Dq_sl.value = 0.50
    P_sl.value = 1.50
    k_sl.value = 2.0

def apply_strong_field():
    Dq_sl.value = 2.20
    P_sl.value = 0.30
    k_sl.value = 2.0

def reset_parameters():
    Dq_sl.value = 1.20
    P_sl.value = 0.80
    k_sl.value = 2.0
    ks_sl.value = 0.80
    kt_sl.value = -0.10
    a3_sl.value = -0.10
    a4_sl.value = 1.20
    Tsoft_sl.value = 0.03
    smooth_chk.value = True
    Q_sl.value = 0.25
    ion_dd.value = "Cu(II)   d9"

weak_btn = widgets.Button(description="Weak-Field Ligand", button_style='info')
strong_btn = widgets.Button(description="Strong-Field Ligand", button_style='warning')
reset_btn = widgets.Button(description="Reset Parameters", button_style='success')

weak_btn.on_click(lambda b: apply_weak_field())
strong_btn.on_click(lambda b: apply_strong_field())
reset_btn.on_click(lambda b: reset_parameters())

preset_controls = HBox([weak_btn, strong_btn, reset_btn], layout=Layout(margin='8px 0 0 0'))
display(VBox([header, row1, acc, panels]))
display(preset_controls)


In [8]:

# --- MP4 Exporter (robust v17) ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.animation import FFMpegWriter
from matplotlib import transforms
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

FONT_SCALE = 1.15   # +15% fonts
TITLE_Y    = 1.02   # common title baseline
WSPACE     = 0.40   # inter-panel spacing

LEVEL_T2G = '#1f77b4'
LEVEL_EG  = '#d62728'

ORBITAL_LATEX = {'dx2-y2': r'd_{x^2-y^2}', 'dz2': r'd_{z^2}', 'dxy': r'd_{xy}', 'dxz': r'd_{xz}', 'dyz': r'd_{yz}'}
ORBITAL_GROUP = {'dx2-y2':'eg', 'dz2':'eg', 'dxy':'t2g', 'dxz':'t2g', 'dyz':'t2g'}

# ---- Exact occupancy enumeration for consistency with Panel 3 ----
ORBITALS = ['dx2-y2','dz2','dxy','dxz','dyz']

def enumerate_occupancies(n):
    occs = []
    # All 5 orbitals, values 0..2, sum == n
    for a in range(3):
        for b in range(3):
            for c in range(3):
                for d in range(3):
                    e = n - (a+b+c+d)
                    if 0 <= e <= 2:
                        occs.append((a,b,c,d,e))
    return occs

def config_energy(occ, energies, P_pair):
    E = 0.0; pairs = 0
    for o, orb in zip(occ, ORBITALS):
        E += energies[orb] * o
        if o >= 2:
            pairs += 1
    return E + P_pair * pairs

def config_degeneracy(occ):
    up = sum(1 for o in occ if o == 1)
    return 2**up

def minimize_electronic_energy(energies, n_electrons, pairing_energy):
    best_E = float('inf'); best_occ = None
    for occ in enumerate_occupancies(int(n_electrons)):
        E = config_energy(occ, energies, pairing_energy)
        if E < best_E:
            best_E, best_occ = E, occ
    occ_dict = {orb:o for o, orb in zip(best_occ, ORBITALS)}
    n_unp = sum(1 for o in best_occ if o == 1)
    return occ_dict, best_E, n_unp

def softmin_electronic_energy(energies, n_electrons, pairing_energy, T_soft):
    if T_soft <= 0:
        return minimize_electronic_energy(energies, n_electrons, pairing_energy)[1]
    E_list = []; g_list = []
    for occ in enumerate_occupancies(int(n_electrons)):
        E_list.append(config_energy(occ, energies, pairing_energy))
        g_list.append(config_degeneracy(occ))
    if not E_list:
        return 0.0
    E0 = min(E_list)
    T = max(T_soft, 1e-12)
    Zp = sum(g*np.exp(-(E - E0)/T) for E,g in zip(E_list, g_list)) + 1e-300
    return E0 - T*np.log(Zp)

def _norm(v):
    v = np.array(v, dtype=float)
    n = np.linalg.norm(v)
    return v/(n if n > 1e-9 else 1.0)

def _perp_basis(v):
    v = _norm(v)
    ref = np.array([0,0,1.0]) if abs(v[2]) < 0.9 else np.array([1.0,0,0])
    u = _norm(np.cross(v, ref))
    w = _norm(np.cross(v, u))
    return u, w

def _cf_energies(Dq, ks, kt, Q):
    Ds = ks * Q
    Dt = kt * Q
    return {
        'dz2':     6*Dq - 2*Ds - 6*Dt,
        'dx2-y2':  6*Dq + 2*Ds - 1*Dt,
        'dxy':    -4*Dq + 2*Ds - 1*Dt,
        'dxz':    -4*Dq - 1*Ds + 4*Dt,
        'dyz':    -4*Dq - 1*Ds + 4*Dt,
    }

def _fill_occupancy_aufbau_hund(e, dcount, P, tol=None):
    # Group energies with tolerance (relative to span)
    Es = list(e.values())
    span = max(Es) - min(Es) if Es else 0.0
    if tol is None:
        tol = max(1e-5, 1e-3*span)
    levels = sorted(e.items(), key=lambda kv: kv[1])  # ascending
    groups = []
    for orb, E in levels:
        if not groups or abs(E - groups[-1][0][1]) > tol:
            groups.append([(orb, E)])
        else:
            groups[-1].append((orb, E))

    occ = {k:0 for k in e}
    total = 0.0
    rem = int(dcount)

    # First pass: singly-occupy within each group (maximize spin)
    for grp in groups:
        if rem <= 0: break
        m = min(len(grp), rem)
        for i in range(m):
            orb = grp[i][0]
            occ[orb] += 1
            total += e[orb]
        rem -= m

    # Second pass: add second electrons (pairing) bottom -> top
    for grp in groups:
        if rem <= 0: break
        for i in range(len(grp)):
            if rem <= 0: break
            orb = grp[i][0]
            if occ[orb] >= 1:
                occ[orb] += 1
                total += e[orb] + P
                rem -= 1

    n_unp = sum(1 for v in occ.values() if v == 1)
    return total, occ, n_unp


def total_energy_and_occ(Dq, P, k, ks, kt, a3, a4, dcount, Q):
    # Prefer using the same mapping functions as Panel 3 if available
    cf = globals().get('crystal_field_energies', None)
    mapfn = globals().get('map_Q_to_DsDt', None)
    if callable(mapfn) and callable(cf):
        Ds, Dt = mapfn(Q, ks=ks, kt=kt)
        e = cf(Dq, Ds, Dt)
    else:
        Ds = ks * Q; Dt = kt * Q
        e = {'dz2': 6*Dq - 2*Ds - 6*Dt,
             'dx2-y2': 6*Dq + 2*Ds - Dt,
             'dxy': -4*Dq + 2*Ds - Dt,
             'dxz': -4*Dq - Ds + 4*Dt,
             'dyz': -4*Dq - Ds + 4*Dt}
    # Exact minimization for occupancy to match interactive panel
    occ_dict, E_el, n_unp = minimize_electronic_energy(e, dcount, P)
    E_lat = a3*Q**3 + a4*Q**4 + k*(Q**2)
    return E_el + E_lat, e, occ_dict, n_unp


def render_atomic(ax, Q, *, metal_color="#3A6EA5", atom_s_factor=0.56, bond_w_factor=0.80,
                  ion_label=None, font_scale=FONT_SCALE, title_y=TITLE_Y):
    ax.cla()
    r_eq, r_ax = 1.0, 1.0 + Q
    donors = np.array([[ r_eq,0,0],[-r_eq,0,0],[0, r_eq,0],[0,-r_eq,0],[0,0, r_ax],[0,0,-r_ax]]) * 1.15
    scale = 2.0
    metal_s    = 260*(scale**2) * 0.65 * atom_s_factor
    carbon_s   = 170*(scale**2) * 0.65 * atom_s_factor
    hydrogen_s =  95*(scale**2) * 0.65 * atom_s_factor
    w_mc       = 2.0*scale * 0.90 * bond_w_factor
    w_ch       = 1.5*scale * 0.90 * bond_w_factor
    COLOR_C = "#444444"; COLOR_BOND_MC = "#888888"; COLOR_BOND_CH = "#aaaaaa"
    COLOR_H_FACE = "#ffffff"; COLOR_H_EDGE = "#999999"

    ax.scatter([0],[0],[0], s=metal_s, c=metal_color, alpha=0.98)
    ax.scatter(donors[:,0], donors[:,1], donors[:,2], s=carbon_s, c=COLOR_C, alpha=0.98)
    for p in donors:
        ax.plot([0,p[0]],[0,p[1]],[0,p[2]], linewidth=w_mc, color=COLOR_BOND_MC, alpha=0.98)

    theta = np.deg2rad(35)
    ring_r = 0.55*np.sin(theta); out_s = 0.55*np.cos(theta)
    for c in donors:
        vhat = _norm(c); u,w = _perp_basis(vhat); center = c + out_s*vhat
        for kidx in range(3):
            ang = 2*np.pi*kidx/3
            p = center + ring_r*(np.cos(ang)*u + np.sin(ang)*w)
            ax.plot([c[0],p[0]],[c[1],p[1]],[c[2],p[2]], linewidth=w_ch, color=COLOR_BOND_CH, alpha=0.98)
            ax.scatter([p[0]],[p[1]],[p[2]], s=hydrogen_s, facecolors=COLOR_H_FACE, edgecolors=COLOR_H_EDGE, linewidths=0.8, alpha=0.98)

    lim = 1.5
    ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim); ax.set_zlim(-lim, lim)
    try: ax.set_box_aspect([1,1,1])
    except Exception: pass
    try: ax.set_axis_off()
    except Exception:
        ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])

    # Title and ion label
    ax.set_title('Molecular Structure', fontsize=int(14*font_scale), y=1.175*title_y)
    if ion_label:
        ax.text2D(0.5, 1.05, str(ion_label), transform=ax.transAxes,
                  ha='center', va='top', fontsize=int(14*font_scale*0.90))

def render_electronic(ax, e, occ, n_unp, *, xlim_max=1.16, label_x_offset=0.024, label_dy_factor=0.11,
                      p2_label_scale=0.90, p2_tick_scale=0.90, p2_perlevel_scale=0.90, font_scale=FONT_SCALE):
    ax.cla()
    P2_LABEL = int(12*font_scale); P2_TICK = int(10*font_scale); P2_PER = int(11*font_scale)
    x_left, x_right = 0.10, 0.70
    order_desc = sorted(e.items(), key=lambda kv: kv[1], reverse=True)
    yvals = [E for _, E in order_desc]
    y_min, y_max = min(yvals), max(yvals)
    yr = (y_max - y_min) if (y_max - y_min) > 1e-9 else 1.0
    pad_bot = 0.12 * yr; pad_top = 0.22 * yr
    ax.set_ylim(y_min - pad_bot, y_max + pad_top)
    ax.set_xlim(-0.05, xlim_max)
    x_label = min(x_right + float(label_x_offset), xlim_max - 0.02)

    for orb, E in order_desc:
        c = LEVEL_EG if ORBITAL_GROUP[orb]=='eg' else LEVEL_T2G
        ax.plot([x_left, x_right], [E, E], color=c, linewidth=2.0)

    base_delta = float(label_dy_factor) * yr
    candidates = [
        ('dx2-y2', e['dx2-y2'], r'$d_{x^2-y^2}\,(b_{1g})$'),
        ('dz2',    e['dz2'],    r'$d_{z^2}\,(a_{1g})$'),
        ('dxy',    e['dxy'],    r'$d_{xy}\,(b_{2g})$'),
        ('dxz_dyz', e['dxz'],   r'$d_{xz},\,d_{yz}\,(e_g)$'),
    ]
    candidates.sort(key=lambda t: t[1], reverse=True)
    y_adj = []; prev = None
    for _, y0, _ in candidates:
        y = y0 if prev is None else min(y0, prev - base_delta)
        y_adj.append(y); prev = y
    for (_, _, txt), ylab in zip(candidates, y_adj):
        ax.text(x_label, ylab, txt, va='center', fontsize=int(P2_PER*p2_perlevel_scale), clip_on=True, ha='left')

    Et = [e['dxy'], e['dxz'], e['dyz']]; Ee = [e['dz2'], e['dx2-y2']]
    y0_t, y1_t = min(Et), max(Et); y0_e, y1_e = min(Ee), max(Ee)
    x_br, tick = x_left - 0.015, 0.015
    ax.plot([x_br, x_br], [y0_t, y1_t], color=LEVEL_T2G, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y0_t, y0_t], color=LEVEL_T2G, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y1_t, y1_t], color=LEVEL_T2G, linewidth=1.2)
    ax.text(x_br - 0.006, 0.5*(y0_t+y1_t), r'$t_{2g}$', ha='right', va='center',
            fontsize=int(P2_LABEL*p2_label_scale), color=LEVEL_T2G, clip_on=True)
    ax.plot([x_br, x_br], [y0_e, y1_e], color=LEVEL_EG, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y0_e, y0_e], color=LEVEL_EG, linewidth=1.2)
    ax.plot([x_br, x_br+tick], [y1_e, y1_e], color=LEVEL_EG, linewidth=1.2)
    ax.text(x_br - 0.006, 0.5*(y0_e+y1_e), r'$e_g$', ha='right', va='center',
            fontsize=int(P2_LABEL*p2_label_scale), color=LEVEL_EG, clip_on=True)

    half   = 0.055 * yr
    dxpair = 0.040
    xbase = {'dx2-y2':0.50,'dz2':0.30,'dxy':0.40,'dxz':0.20,'dyz':0.60}
    for orb, E in order_desc:
        n = occ.get(orb, 0)
        xb = xbase.get(orb, 0.40)
        if n == 1:
            ax.annotate('', xy=(xb, E + half), xytext=(xb, E - half), arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))
        elif n == 2:
            xL, xR = xb - dxpair/2.0, xb + dxpair/2.0
            ax.annotate('', xy=(xL, E + half), xytext=(xL, E - half), arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))
            ax.annotate('', xy=(xR, E - half), xytext=(xR, E + half), arrowprops=dict(arrowstyle='-|>', lw=1.2, color='black'))

    ax.set_yticks(sorted(set([round(v,3) for v in e.values()])))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.set_xticks([])
    ax.set_ylabel('Energy (eV)', fontsize=int(P2_LABEL*p2_label_scale))
    ax.set_xlabel(f'Levels + Spins (unpaired = {n_unp})', fontsize=int(P2_LABEL*p2_label_scale))
    ax.tick_params(axis='both', labelsize=int(P2_TICK*p2_tick_scale))
    ax.set_title('Electronic Structure', fontsize=int(14*font_scale), y=TITLE_Y)

    bt = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    ax.text(x_right + float(label_x_offset), 0.92, r'$D_{4h}$', transform=bt, ha='left',
            fontsize=int(P2_LABEL*p2_label_scale), fontweight='bold')
    ax.text(x_br, 0.92, r'$O_h$', transform=bt, ha='center',
            fontsize=int(P2_LABEL*p2_label_scale), fontweight='bold')

def render_stability(ax, Qs, E_curve, q_now, E_now, *, font_scale=FONT_SCALE, title_y=TITLE_Y):
    ax.cla()
    ax.plot(Qs, E_curve, lw=2.0)
    ax.plot([q_now], [E_now], 'o', ms=6)
    ax.set_xlabel('Distortion Q', fontsize=int(12*font_scale))
    ax.set_ylabel('Total Energy (arb.)', fontsize=int(12*font_scale))
    ax.set_title('Stability', fontsize=int(14*font_scale), y=title_y)

def _read_params_from_widgets():
    # Attempt to pull current UI state; fallback to defaults
    try:
        par = dict(Dq=float(Dq_sl.value), P=float(P_sl.value), k=float(k_sl.value),
                   ks=float(ks_sl.value), kt=float(kt_sl.value),
                   a3=float(a3_sl.value), a4=float(a4_sl.value),
                   dcount=int(dcount_sl.value))
        Q_min, Q_max = float(Q_sl.min), float(Q_sl.max)
        # Ion label
        ion_label = None
        for cand in ['ion_dd','ion_sel','ion_dropdown','metal_ion_dd','tm_dd']:
            if cand in globals():
                ion_label = globals()[cand].value
                break
        return par, Q_min, Q_max, ion_label
    except Exception:
        # Safe defaults
        par = dict(Dq=1.20, P=0.80, k=2.0, ks=0.80, kt=-0.10, a3=-0.10, a4=1.20, dcount=9)
        return par, -0.6, 0.6, None

def export_jt_video(path, include_atomic=True, fps=30, seconds=15,
                                   xlim_max=1.16, label_x_offset=0.024, label_dy_factor=0.11,
                                   p2_label_scale=0.90, p2_tick_scale=0.90, p2_perlevel_scale=0.90,
                                   metal_color="#3A6EA5", atom_s_factor=0.56, bond_w_factor=0.80,
                                   wspace=WSPACE, font_scale=FONT_SCALE, title_y=TITLE_Y):
    par, Q_min, Q_max, ion_label = _read_params_from_widgets()
    nframes = int(fps * seconds)
    Qs = np.linspace(Q_min, Q_max, nframes)
    Qs_curve = np.linspace(Q_min, Q_max, 400)
    smooth_enabled = False; T_soft = 0.0
    try:
        smooth_enabled = bool(smooth_chk.value)
        T_soft = float(Tsoft_sl.value)
    except Exception:
        pass
    E_curve = _energy_curve_from_params(par, Qs_curve, smooth_enabled=smooth_enabled, T_soft=T_soft)

    fig = plt.figure(figsize=(16.5, 5.1), constrained_layout=False)
    gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 1])
    ax1 = fig.add_subplot(gs[0, 0], projection='3d') if include_atomic else None
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    fig.subplots_adjust(wspace=wspace)

    writer = FFMpegWriter(fps=fps, metadata=dict(artist='JT-Notebook'), bitrate=2400)
    with writer.saving(fig, path, dpi=160):
        for q in Qs:
            E_now, e, occ, n_unp = total_energy_and_occ(**par, Q=q)
            if include_atomic:
                render_atomic(ax1, q, metal_color=metal_color, atom_s_factor=atom_s_factor, bond_w_factor=bond_w_factor,
                              ion_label=(ion_label if ion_label else f'd^{par.get("dcount", 0)}'),
                              font_scale=font_scale, title_y=title_y)
            render_electronic(ax2, e, occ, n_unp,
                              xlim_max=xlim_max, label_x_offset=label_x_offset, label_dy_factor=label_dy_factor,
                              p2_label_scale=p2_label_scale, p2_tick_scale=p2_tick_scale, p2_perlevel_scale=p2_perlevel_scale,
                              font_scale=font_scale)
            render_stability(ax3, Qs_curve, E_curve, q, E_now, font_scale=font_scale, title_y=title_y)
            writer.grab_frame()
    plt.close(fig)
    print("Saved:", path)

def _energy_curve_from_params(par, Qs_curve, smooth_enabled=False, T_soft=0.0):
    cf = globals().get('crystal_field_energies', None)
    mapfn = globals().get('map_Q_to_DsDt', None)
    use_global = callable(cf) and callable(mapfn)
    E_vals = []
    for Q in Qs_curve:
        if use_global and smooth_enabled:
            Ds, Dt = mapfn(Q, ks=par['ks'], kt=par['kt'])
            e = cf(par['Dq'], Ds, Dt)
            E_el = softmin_electronic_energy(e, par['dcount'], par['P'], T_soft)
        elif use_global and not smooth_enabled:
            Ds, Dt = mapfn(Q, ks=par['ks'], kt=par['kt'])
            e = cf(par['Dq'], Ds, Dt)
            _, E_el, _ = minimize_electronic_energy(e, par['dcount'], par['P'])
        else:
            # Fallback to local mapping + minimization
            E_el, _, _, _ = total_energy_and_occ(**par, Q=Q)
            # total_energy_and_occ includes lattice already; but for curve we want total energy:
            E_vals.append(E_el)
            continue
        # Add lattice part
        E_lat = par['a3']*Q**3 + par['a4']*Q**4 + par['k']*(Q**2)
        E_vals.append(E_el + E_lat)
    return np.array(E_vals)

# Call export_jt_video after all functions are defined, uncomment for use:
"""export_jt_video(
    "jt_demo.mp4",
    include_atomic=True,
    fps=30,
    seconds=10,
    xlim_max=1.16,
    label_x_offset=0.024,
    label_dy_factor=0.07,
    p2_label_scale=1.0,
    p2_tick_scale=1.0,
    p2_perlevel_scale=1.00,
    metal_color="#3A6EA5",
    atom_s_factor=0.56,
    bond_w_factor=0.80,
)"""

'export_jt_video(\n    "jt_demo.mp4",\n    include_atomic=True,\n    fps=30,\n    seconds=10,\n    xlim_max=1.16,\n    label_x_offset=0.024,\n    label_dy_factor=0.07,\n    p2_label_scale=1.0,\n    p2_tick_scale=1.0,\n    p2_perlevel_scale=1.00,\n    metal_color="#3A6EA5",\n    atom_s_factor=0.56,\n    bond_w_factor=0.80,\n)'


## In‑class mini‑project (15–20 minutes)

**Goal:** Diagnose Jahn–Teller activity and predict distortion preferences using the dashboard.

1. **Cu(II) d<sup>9</sup> (default).**
   - Set $Q=0$, record the level ordering and occupancies.
   - Slowly increase $Q>0$ to about $+0.3$.  
     **Tasks:**  
     (a) Which orbital moves down most strongly?  
     (b) Does the electron configuration change at small $Q$?  
     (c) Is the minimum of $E(Q)$ at $Q>0$ (elongation) or $Q<0$ (compression)?
2. **Mn(III) d<sup>4</sup> (high‑spin).**
   - Switch ion to **Mn(III) d4**, hold $P$ near the default (high‑spin).  
     **Tasks:**  
     (a) Identify the JT‑active subshell.  
     (b) Predict elongation vs compression; verify by scanning $Q$.  
     (c) Compare stabilization $\Delta E$ to Cu(II). Why might magnitudes differ?
3. **JT‑inactive references.**
   - Try **Cr(III) d3** and **Fe(III) d5 (high‑spin)**.  
     **Tasks:** Explain why little/no JT driving force appears (hint: either no degeneracy or half‑filled symmetry).
4. **Parameter sensitivity (optional).**
   - Increase **$k$** (stiffer lattice): what happens to the double‑well depth and optimal $Q^\ast$?  
   - Adjust **$a_3$** to skew wells: can you favor elongation over compression with the same electronic filling?
5. **Short write‑up.** In 3–5 sentences, summarize which ions are JT‑active, the sign of distortion, and how $k$ and $P$ shape the outcome.

> **Deliverable:** a screenshot of the three panels at a chosen $Q^\ast$ for one JT‑active ion (include your settings), plus short answers to the bolded prompts.


### References & Further Reading

**Classic papers**
- H. A. Jahn & E. Teller, *Stability of Polyatomic Molecules in Degenerate Electronic States. I. Orbital Degeneracy*, **Proc. R. Soc. A** 161, 220–235 (1937). DOI: https://doi.org/10.1098/rspa.1937.0142  
- Y. Tanabe & S. Sugano, *On the Absorption Spectra of Complex Ions. I*, **J. Phys. Soc. Jpn.** 9, 753–766 (1954). DOI: https://doi.org/10.1143/JPSJ.9.753  
- Y. Tanabe & S. Sugano, *On the Absorption Spectra of Complex Ions. II*, **J. Phys. Soc. Jpn.** 9, 766–779 (1954). DOI: https://doi.org/10.1143/JPSJ.9.766  

**Monographs & textbooks**
- I. B. Bersuker, *The Jahn–Teller Effect* (Cambridge Univ. Press, 2006). DOI: https://doi.org/10.1017/CBO9780511524769  
- R. Englman, *The Jahn–Teller Effect in Molecules and Crystals* (Wiley‑Interscience, 1972).  
- I. B. Bersuker & V. Z. Polinger, *Vibronic Interactions in Molecules and Crystals* (Springer, 1989; softcover reprint 2012). DOI: https://doi.org/10.1007/978-3-642-83479-0  
- S. Sugano, Y. Tanabe & H. Kamimura, *Multiplets of Transition‑Metal Ions in Crystals* (Academic Press, 1970).  
- C. J. Ballhausen, *Introduction to Ligand Field Theory* (McGraw‑Hill, 1962).  
- A. B. P. Lever, *Inorganic Electronic Spectroscopy*, 2nd ed. (Elsevier, 1984).  

**Modern reviews and tutorials**
- I. B. Bersuker, *Pseudo‑Jahn–Teller Effect—A Two‑State Paradigm in Formation, Deformation, and Transformation of Molecular Systems and Solids*, **Chem. Rev.** 113, 1351–1390 (2013). DOI: https://doi.org/10.1021/cr300279n  
- I. B. Bersuker, *Jahn–Teller and Pseudo–Jahn–Teller Effects: From Particular Features to General Tools in Exploring Molecular and Solid‑State Properties*, **Chem. Rev.** 121, 1463–1512 (2021). DOI: https://doi.org/10.1021/acs.chemrev.0c00718  
- H. Köppel, *Theory of the Jahn–Teller Effect* (overview chapter), Wiley (2011). DOI: https://doi.org/10.1002/9780470749593.hrs060  

**Practical context**
- For ligand‑field conventions and the $10D_q$ splitting used in this notebook, see Lever (1984) and Ballhausen (1962).